# **사전 준비**

In [ ]:
!pip install langchain langchain-neo4j langchain-openai langchain_community

In [ ]:
from langchain_neo4j import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from dotenv import load_dotenv

load_dotenv("/content/.env")

# 실제 인스턴스 정보를 입력한다.

NEO4J_URI = "neo4j+s://..."
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

embedding = OpenAIEmbeddings()

# Neo4j 그래프 객체 생성
graph = Neo4jVector.from_existing_graph(
    embeding=embedding,
    node_label="__Entity__",
    text_node_properties=["description"],
    embedding_node_property="embedding",
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Neo4j Graph 객체 추가 생성 (Cypher 쿼리 실행용)
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4JPASSWORD,
    database=NEO4J_DATABASE
)

# **로컬 검색 구현**

In [ ]:
def fetch_entity_context(entity_name):
  context = {"name": entity_name}

  try:
    # 텍스트 청크 가져오기
    chunk_query = """
    MATCH (e:__Entity__ {name: $entity_name})<-[:HAS_ENTITY]-(c:__Chunk__)
    RETURN c.text AS text
    """

    chunk_result = neo4j_graph.query(chunk_query, {"entity_name": entity_name})
    context["text_chunks"] = [r["text"] for r in chunk_result] if chunk_result
    else ["No text chunk available"]

    # 커뮤니티 보고서 가져오기
    community_query = """
    MATCH (e:__Entity__ {name: $entity_name})-[:IN_COMMUNITY]->(com:__Community__)
    RETURN com.full_content AS report
    """

    community_result = neo4j_graph.query(community_query, {"entity_name": entity_name})
    context["community_reports"] = [r["report"] for r in community_result] if
    community_result else ["No Community report available"]

    # 관련 엔티티 가져오기
    related_query = """
    MATCH (e:__Entity__ {name: $entity_name})-[:RELATED]->(related:__Entity__)
    RETURN related.name AS name, related.description AS description
    """

    related_result = neo4j_graph.query(related_query, {"entity_name": entity_name})
    context["related_entities"] = (
        [{"name": r["name"], "description": r["description"]} for r in related_result]
        if related_result else []
    )
  except Exception as e:
    context["error"] = f"Error fetching context: {str(e)}"

  return context

In [ ]:
def create_structured_context(all_contexts, query):
  context_str = "## 질문과 관련된 엔티티 정보\n\n"
  context_str += "아래는 질문에 답변하는 데 유용한 엔티티들의 구조화된 정보입니다:\n\n"

  for i, ctx in enumerate(all_contexts, 1):
    context_str += f"### 엔티티 {i}: {ctx['name']}\n"
    context_str += f"- **설명**: {ctx['description']}\n"
    context_str += "- **텍스트 청크"